410921202 資工四 林芷萱 0605 Practice

In [1]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# 識別手勢
def recognize_gesture(landmarks):
    thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP].y # 拇指
    index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP].y #食指
    middle_tip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y # 中指
    ring_tip = landmarks[mp_hands.HandLandmark.RING_FINGER_TIP].y # 無名指
    pinky_tip = landmarks[mp_hands.HandLandmark.PINKY_TIP].y # 小指

    # 石頭:拇指比食指、中指、無名指、小指都低
    if thumb_tip < index_tip and thumb_tip < middle_tip and thumb_tip < ring_tip and thumb_tip < pinky_tip:
        return "Rock"
    # 布:拇指比食指、中指、無名指、小指都高
    elif thumb_tip > index_tip and thumb_tip > middle_tip and thumb_tip > ring_tip and thumb_tip > pinky_tip:
        return "Paper"
    # 剪刀:食指、中指高於拇指、無名指、小指
    elif index_tip < thumb_tip and middle_tip < thumb_tip and index_tip < ring_tip and index_tip < pinky_tip and middle_tip < ring_tip and middle_tip < pinky_tip:
        return "Scissor"
    # 其他
    else:
        return "Unknown"

cap = cv2.VideoCapture(0)
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

# confidence要大於0.7
with mp_hands.Hands(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7) as hands:

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # 使用mp_drawing.draw_landmarks來畫手landmark
        # 使用recognize_gesture函數來識別手勢
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        gesture = "None" # 初始是None
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                gesture = recognize_gesture(hand_landmarks.landmark)
                
        cv2.putText(image, f'{gesture}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.imshow('MediaPipe Hands', image)
        
        if cv2.waitKey(1) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()
